In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
import numpy as np

model_name = 'distilbert-base-uncased'
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def preprocess_text(text):
    encodings = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
    return encodings

In [ ]:
emotion_labels = ["happy", "sad", "angry", "anxious", "neutral", "excited"]

In [ ]:
def predict_emotion(text):
    inputs = preprocess_text(text)
    logits = model(inputs.input_ids).logits
    predicted_class = tf.argmax(logits, axis=1).numpy()[0]
    return emotion_labels[predicted_class]

In [ ]:
def generate_response(emotion):
    responses = {
        "happy": "I'm glad you're feeling good! 😊 What's making you so happy?",
        "sad": "I'm sorry to hear that. Want to talk about what's on your mind? 💙",
        "angry": "I understand that you're upset. Take a deep breath. 😌",
        "anxious": "It's okay to feel anxious. Would you like to share more? 💭",
        "neutral": "I see you're feeling neutral. What's going on? 🤔",
        "excited": "That’s awesome! Tell me more about what’s got you excited! 🎉"
    }

    return responses.get(emotion, "I’m here to listen! 😊")

user_input = "I'm feeling a bit overwhelmed today."
predicted_emotion = predict_emotion(user_input)
response = generate_response(predicted_emotion)
print(f"Emotion: {predicted_emotion}")
print(f"Response: {response}")

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_texts = ["I am so happy today!", "I feel very sad.", "I am really angry."]
train_labels = [0, 1, 2]

train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="tf")
train_labels = tf.convert_to_tensor(train_labels)

history = model.fit(
    x=train_encodings.input_ids,  
    y=train_labels,      
    epochs=3,           
    batch_size=2
)